In [6]:
#!/usr/bin/python3

import cobra 
import pandas as pd
import re
import sys
import getopt
import os.path
import copy
import csv
import math
import cobra.flux_analysis.variability
import subprocess
import shutil, errno
import statistics
from cobra import Reaction, Metabolite
import cometspy as c
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams.update({'figure.max_open_warning': 0})
#from importExcelModel import *

def old_version_model_to_new(model):
    modeltmp=model.copy()
    for metabolite in modeltmp.metabolites:
        metabolite.id = re.sub('\[c\]$',r'_c',metabolite.id)
        metabolite.id = re.sub('\[p\]$',r'_p',metabolite.id)
        metabolite.id = re.sub('\[e\]$',r'_e',metabolite.id)
    print("Parsing metabolites...")
    modeltmp.repair()
    cobra.io.write_sbml_model(modeltmp,'./tmp.xml')
    modeltmp=cobra.io.read_sbml_model('./tmp.xml')
    for rxn in modeltmp.reactions:
        rxn.id = re.sub('\(p\)$',r'_p',rxn.id)
        rxn.id = re.sub('\(c\)$',r'_c',rxn.id)
        rxn.id = re.sub('\(e\)$',r'_e',rxn.id)
        # To solve possible problems in changing names
    modeltmp.repair()
    print("Parsing reactions...")
    cobra.io.write_sbml_model(modeltmp,'./tmp.xml')
    modeltmp=cobra.io.read_sbml_model('./tmp.xml')
    os.remove("./tmp.xml")
    return(modeltmp)
           

def add_monitor_reaction(model, target_metabolite, syntax='['):
    configured_model = model.copy()
    #add an artificial reaction to monitor target production:
    target_c = configured_model.metabolites.get_by_id(target_metabolite)
    #check if metabolite exists in extracellular compartment
    met_to_monitor_id = target_c.id.replace(syntax+'c', syntax+'e')
    monitor_exists = met_to_monitor_id in [m.id for m in configured_model.metabolites]
    
    if monitor_exists:
        
        print('No need to add monitor reaction to model')
    
    else:
        
        print('Adding monitoring reaction to model...')
        
        target_e = Metabolite( met_to_monitor_id,
                               formula= target_c.formula,
                               name=target_c.name,
                               compartment='e')
        model.add_metabolites([target_e])
        monitor_rxn = Reaction(target_e.id+'_MONITOR_RXN')
        monitor_rxn.name = target_e.name+'monitor reaction'
        monitor_rxn.lower_bound = 0.
        monitor_rxn.upper_bound = 1000.

        monitor_rxn.add_metabolites({
            target_c : -1,
            target_e : 1
        })

        configured_model.add_reactions([monitor_rxn])
        
        configured_model.add_boundary(configured_model.metabolites.get_by_id(target_e.id), type="exchange")

    return configured_model

def initialize_models(model,tmp_id, WT, target_metabolite, modifications=dict()):
    configured_model = model.copy()
    
    configured_model = add_monitor_reaction(configured_model, target_metabolite[0], syntax='_')
    configured_model = add_monitor_reaction(configured_model, target_metabolite[1], syntax='_')
    #configured_model = add_monitor_reaction(configured_model, target_metabolite[2], syntax='_')
    #configured_model = add_monitor_reaction(configured_model, target_metabolite[3], syntax='_')

    #produce sink rection by changing direction:
    configured_model.reactions.get_by_id(target_metabolite[1].replace('_c','_e')+'_MONITOR_RXN').bounds = (-1000, 0)
    configured_model.reactions.get_by_id('EX_'+target_metabolite[1].replace('_c','_e')).bounds = (-10, 0)
    configured_model = model_modifications(configured_model, modifications)
    cobra.io.write_sbml_model(configured_model, 'pruebaKT.xml')

    
    if WT==True:
        modelcomets = c.model(configured_model)
        modelcomets.id=tmp_id
        c.model.write_comets_model(modelcomets)
        
    else:
        modelcomets = c.model(configured_model)
        modelcomets.id='model_tmp2'
        c.model.write_comets_model(modelcomets)
        
    return modelcomets
        
def model_modifications(model, bounds_to_change):
    # Single GEMs parameter modifications
    #You can change the bounds of a reaction
    for r, bs in bounds_to_change.items():
        model.reactions.get_by_id(r.id).bounds = bs
    
    print('Modifications of the model done!')

    return model
    
    
def initialize_layout(filename):
    #The layout is a file with the stablished format
    layout=c.layout(filename)
    #if you need to optimize biomass, its value is changed here 
    #layout.initial_pop=[[0.0,0.0,float(biomass1),float(biomass2)]]
    return layout

def initialize_params(package, globall):
    """Function to initialize the comets' params class
    it can be initialize by submitting two files, one for the package parameters
    and one for the global ones.
    If you don't submit a file, the params class will be initialize with the values stated below
    which have been tested in this simulation"""
    
    if package and globall is not None:
        params = c.params(global_params = globall, package_params= package)
    elif package is not None:
        params = c.params(package_params=package)
    elif globall is not None:
        params = c.params(global_params=globall)
        
    else:
        params = c.params()
        params.all_params['maxCycles']=750
        params.all_params['timeStep']=0.1
        params.all_params['spaceWidth']=0.05
        params.all_params['allowCellOverlap']= True
        params.all_params['deathRate']= 0.0
        params.all_params['numRunThreads']= 8
        params.all_params['maxSpaceBiomass']= 1000
        params.all_params['defaultVmax']=20
        params.all_params['showCycleTime']=True
        params.all_params['writeTotalBiomassLog']=True
        params.all_params['writeMediaLog']=True
        params.all_params['writeFluxLog']=True
        params.all_params['useLogNameTimeStamp']=False
        params.all_params['FluxLogRate']=1
        params.all_params['MediaLogRate']=1
        params.all_params['exchangestyle']='Standard FBA'

        
    #check if param 'maxSpaceBiomass' has default value
    if (params.all_params['maxSpaceBiomass']== 0.1):
        print('The parameter "maxSpaceBiomass" is set to 0.1.\n' \
              'It may need to change if the mo used growths well.')

    return params

def make_df(strains,metabolites, comets,params,name=None, wt=True):
    '''This function creates a figure and saves it to pdf format.
    It also creates the file biomass_vs_met.txt which contais the quantity
    of each strain and metabolite and has the following columns:
    time(h), strain1 ... strainX, met1 ... metX.'''
    if name==None:
        file_name='_'.join(metabolites)
    else:
        file_name=name
    df = comets.media #We get the media composition results'
    df2=comets.total_biomass #We get the biomass results
    columns=['cycle']
    for i in range(0,len(strains)):
        columns.append(strains[i])
    df2.columns=columns
    max_cycles=params.all_params['maxCycles']
    """For each metabolite a column with all zeros is added to the dataframe and each row that contains a value
     (metabolite concentration)is changed in the dataframe"""
    for d in metabolites:
        met =df[df['metabolite'] == d]
        print(met)
        temp=np.zeros(max_cycles+1) #Create an array with all zeros
        df2[d]=temp #We added it to the dataframe
        j=1
        while j < (max_cycles+1): #For each cycle
            if (met.cycle==j).any(): #If the row exists
                df2.loc[j-1,d] = float(met[met['cycle']==j]['conc_mmol']) #Its dataframe value is changed
            j+=1  
    if wt==True:
        np.savetxt(r'biomass_vs_'+file_name+'_template_WT.csv', df2.values, fmt='%s',delimiter='\t')
    else:
        np.savetxt(r'biomass_vs_'+file_name+'_template.csv', df2.values, fmt='%s',delimiter='\t')#The data is saved
    return df2

def make_graph(strains,metabolites, df2, GR, yield_, maxBiomass, df3, name=None):
    if name==None:
        file_name='_'.join(metabolites)
    else:
        file_name=name
    #---Starting the figure 
    plt.ioff()
    fig, ax = plt.subplots()

    ax.set_xlabel('time (h)')
    ax.set_ylabel('biomass (g/L)')
    c=['teal', 'indianred', '#93c5bb', '#eecaa6']
    for i in strains:
        ax.plot(df2['cycle']*0.1, df2[i], label='PET4', color=c[0])
        ax.plot(df3['cycle']*0.1, df3[i], label='WT', color=c[1])
    ax2 = ax.twinx()
    ax2.set_ylabel('metabolite conc (mM)')
    for m in metabolites:
        ax2.plot(df2['cycle']*0.1, df2[m], label=m+'_PET4', color=c[2])
        ax2.plot(df3['cycle']*0.1, df3[m], label=m+'_WT', color=c[3])
    handles, labels = ax.get_legend_handles_labels()
    handle_list, label_list = [], []

    for handle, label in zip(handles, labels):
        if label not in label_list:
            handle_list.append(handle)
            label_list.append(label)
    handles, labels = ax2.get_legend_handles_labels()
    for handle, label in zip(handles, labels):
        if label not in label_list:
            handle_list.append(handle)
            label_list.append(label)
    plt.legend(handle_list, label_list,bbox_to_anchor=(0.6,0.2))
    
    data = [['E.coli_glucose',round(GR[0],2), round(yield_[0],2), round(maxBiomass[0],2)],
            ['E.coli_WT',round(GR[1],2), round(yield_[1],2), round(maxBiomass[1],2)]]
    table = plt.table(cellText=data, colLabels=['Strain','Growth Rate (µmax)', 'Biomass Yield', 'Final Biomass (g/L)'], loc='bottom', 
                  cellLoc='center', colColours=['white', 'white', 'white', 'white'], bbox=[0.0,-0.5,1,.28])
    #saving the figure to a pdf
    plt.subplots_adjust(left=0.2, bottom=0.2)
    #plt.show()
    plt.tight_layout()
    plt.savefig('biomass_vs_'+file_name+'_template_plot.pdf')
    return df2

def end_simulation_cycle(met, strains, df2):
    """function that stablishes the endCyle after a certain stop condition
    If the initial biomass of a microorganism needs to be used, you can access it with
    float(df2.at[0,'Ecoli1']), where df2 is the dataframe created in make_graph() function.
    The following is a example of a stop condition in which the cycle is stopped when either:
    glucose and acetate are exhausted or when glucose is exhausted and the microorganisms
    can't use acetate to grow.
    The stop condition should be suitable for your simulation.
    """
    
    iniBiomass=float(df2.at[0,strains[0]])
    totMet=float(df2.at[0,met])
    #totMet2=float(df2.at[0,met[1]])
    endCycle=0
    for row in df2.iterrows():
        endCycle=int(row[1][0])
        metConc=float(row[1][2])
        if totMet==0: #This is an example that stops the simulation if concentrations of
            endCycle=endCycle
            break
        
    return iniBiomass, endCycle, totMet#, totMet2

def biomass_yield(df2,endCycle, strains):
    # Function that compute final biomass as the maximum biomass of each strain.
    
    finalBiomass={}
    for i in range(1, len(strains)+1):
        finalBiomass['finalBiomass'+str(i)]=0
        count=0
        for row in df2.iterrows():
            if float(row[1][i])>finalBiomass['finalBiomass'+str(i)]:
                finalBiomass['finalBiomass'+str(i)]=float(row[1][i])
            if count > endCycle:
                break
            count+=1
        total_finalBiomass=sum(finalBiomass.values())
    return total_finalBiomass
    

def compute_fitness(MaximumYield, finalBiomass, iniBiomass, fitFunc, fitTime, comets,layout,model,df2):
    """Establising a fitness function is one of the most critical steps in FLYCOP. There needs to be carefully
    thinked. Here are some examples that can be totally rewritable"""
    if(fitFunc=='Yield'):
        fitness=(biomassYieldNew/MaximumYield) # Normalizing Yield
    elif(fitFunc=='MaxYield_MinTime'):
        fitness=0.5*(biomassYieldNew/MaximumYield)+0.5*fitTime #Normalizing yield
    elif(fitFunc=='YieldNewScattered'): # (biomass^4)*10: To spread values from ~0.45-0.55 values to 0.5 to 1
        fitness=(biomassYieldNew**4)*10
    elif(fitFunc=='MaxYieldNewScattered_MinTime'):
        fitness=0.5*((biomassYieldNew**4)*10)+0.5*fitTime
    elif(fitFunc=='Biomass'):
        fitness=float(finalBiomass-iniBiomass)        
    elif(fitFunc=='MaxBiomass_MinTime'):
        fitness=0.5*(float(finalBiomass-iniBiomass))+0.5*fitTime
    elif((fitFunc=='GR')or(fitFunc=='MaxGR_MinTime')):
        flux = comets.fluxes
        gr=[]
        objective = model.objective
        gr.append(flux.iat[2,int(objective+3)])
        fitGR=sum(gr)/len(gr)
    if fitFunc == 'GR':
        fitness=fitGR
    elif(fitFunc=='MaxGR_MinTime'):
        fitness=0.5*fitGR+0.5*fitTime
    return fitness

def calculate_uptake(comets,layout, met):
    uptakeMet=[]
    flux=comets.fluxes
        #Compute metabolite uptake
    model=layout.models[0]
    df3 = model.reactions
    for me in met:
        j=0
        numRxnMet=int(df3.loc[df3['REACTION_NAMES']=='EX_'+str(me)]['ID'])
        uptakeMet.append(float(flux.iat[j+2,numRxnMet+3]))
        j+=1
    return uptakeMet


def Flycop(filename='putida_PET', fitFunc='GR', dirPlot='', repeat=2,params_package=None, params_global=None):
    """Primarily function of the simulation
    You need to add to it the parameters to adjust.
    You need to fill these variables below(layout, model_id, model, wgMet, metabolites, strains) with your information
    layout= The layout for the comets simulation
    model_id = the names of the models in your layout
    model= the name of the model yours modifications are from
    wgMet=molecular weight of the metabolite to compute fitness. Must be divided by 1000.
    metabolites= list with the names of the metabolites to study
    strains = list with the names of the strains you're working with
    simulationID = will be part of the filename for the results 
    metUptake = the metabolite you want to calculate the uptake of"""
    

    model_file='configured_model.mat'
    layout_file_wt='layoutWT.txt'
    layout_file='layoutPET4.txt'
    filename_wt='WT'
    model_id='model_tmp1' #These are the model names that have to be in the layout
    wgMet=[0.1922] #molecular weigh of the metabolite that you want to calculate the uptake of
    metabolites=['pet_e', 'glycol_e', 'tpha_e', 'C80aPHA_e'] #Metabolites to plot. Metabolite names 
    metabolite_to_create=['C80aPHA_c', 'PHAg_c','5drib_c', 'amob_c']             #as they are written in the layout
    strains=['P.putida KT2440'] #Name of the strains - going to apper in the graph
    metUptake = ['pet_e']
    
    if not(os.path.exists('CometsResults')):
        os.makedirs('CometsResults')
    
    totfitness=0
    sumTotBiomass=0
    sumTotYield=0
    totfitness_wt=0
    sumTotBiomass_wt=0
    sumTotYield_wt=0
    fitnessList=[]
    fitnessList_wt=[]
    print('Initilazing models...')
    all_gdls = ['ABUTD','ALAR','ALATA_L','FORGLUIH2','ICDHyr','ICL','MALS','MCITL2','OARGDC','ORNCD','PDHbr','PDHcr','PPCSCT','RBK','SUCOAS','UPPN','DM_C80aPHA','sink_PHAg']
    minimum_gdls = ['ORNCD','ICDHyr','ICL','MALS','sink_PHAg','DM_C80aPHA' ]
    model = cobra.io.load_matlab_model('configured_model.mat')
    model = old_version_model_to_new(model)
    minimum_modifications = { model.reactions.get_by_id(r) : (0,0) for r in minimum_gdls }
    modelcomets_wt=initialize_models(model, model_id, True, metabolite_to_create, modifications=minimum_modifications)    
    pet4_modifications = { model.reactions.get_by_id(r) : (0,0) for r in all_gdls }
    modelcomets=initialize_models(model, model_id, False, metabolite_to_create, modifications=pet4_modifications)
    

    print('Initialazing layout...')
    layout_wt= initialize_layout(layout_file_wt)
    layout= initialize_layout(layout_file)
    print('Done!')
    print('Initialazing params...')

    params =initialize_params(params_package, params_global)
   
    #establish comets class
    comets_wt = c.comets(layout_wt, params)
    comets=c.comets(layout, params)   

    # To repeat X times, due to random behaviour in COMETS:
    for i in range(repeat):

        comets.run(delete_files=False)
        
        #make the graphic
        df2=make_df(strains,metabolites, comets, params,filename, wt=False)
        # 7.- Compute fitness (measure to optimize):
        print('computing fitness...')
        
        # 7.1.- Determine endCycle: when glucose and acetate are exhausted
        iniBiomass, endCycle, totMetSucr = end_simulation_cycle(metabolites[0], strains,df2)
        
        # 7.2.- Compute first element fitness: maximize biomass yield
        finalBiomass =biomass_yield(df2,endCycle, strains)
        if wgMet is not None:
            #totmet=((totMetSucr*wgMet[0])+(totMetGlc*wgMet[1])+(totMetmal*wgMet[2])+(totMetFru*wgMet[3])+(totMetLac*wgMet[4]))
            biomassYieldNew=float(finalBiomass-iniBiomass)/((totMetSucr*wgMet[0]))#+(totMet2*wgMet[1])) # molecular weigth of met per mmol
        # For normalizing yield
        MaximumYield=1
        
        # 7.3.- Compute second element fitnes: minimize time        
        fitTime=1-(float(endCycle)/float(params.all_params['maxCycles']))
        
        # 7.4.- Compute joint fitness, as a 50% each element.
        fitness= compute_fitness(MaximumYield, finalBiomass, iniBiomass, fitFunc, fitTime,
                                 comets, layout,modelcomets,df2)
        # 7.5.- Calculate the uptake of metabolite
        uptakeMet =calculate_uptake(comets,layout, metUptake)
            
            
        print(" Total biomass: "+str(round(finalBiomass,6))+" in cycle "+str(endCycle)+". Biomass yield="+str(round(biomassYieldNew,6)))
   
        totfitness=totfitness+fitness
        fitnessList.append(fitness)
        sumTotBiomass=sumTotBiomass+finalBiomass
        sumTotYield=sumTotYield+biomassYieldNew
            
        # Copy individual solution
        #x= '_'.join(metabolites)
        x=filename
        y='_'.join(metUptake)
        file='IndividualRunsResults/'+'biomass_vs_'+x+'_run'+str(i)+'_'+str(fitness)+'_'+str(endCycle)+'.pdf'        
            
    avgfitness=totfitness/repeat
    sdfitness=statistics.stdev(fitnessList)
    avgBiomass=sumTotBiomass/repeat
    avgYield=sumTotYield/repeat
    
    print("Fitness_function\tconfiguration\tfitness\tsd\tavg.Biomass\tavg.Yield\tendCycle\tUptakeMet")
                    
    print(fitFunc+"\t"+str(filename)+'\t'+','+y+"\t"+str(round(avgfitness,6))+"\t"+str(sdfitness)+"\t"+str(round(avgBiomass,6))+"\t"+str(round(avgYield,6))+"\t"+str(endCycle)+"\t"+str(uptakeMet))
    with open(dirPlot+"configurationsResults"+fitFunc+".txt", "a") as myfile:
        myfile.write(fitFunc+"\t"+str(filename)+'\t'+','+"\t"+str(round(avgfitness,6))+"\t"+str(sdfitness)+"\t"+str(round(avgBiomass,6))+"\t"+str(round(avgYield,6))+"\t"+str(endCycle)+"\t"+str(uptakeMet)+"\n")
  
    print("Avg.fitness(sd):\t"+str(avgfitness)+"\t"+str(sdfitness)+"\n")
    if(sdfitness>0.1):
        avgfitness=0.0
    
    
    for i in range(repeat):

        comets_wt.run(delete_files=False)

        #make the graphic
        df2_wt=make_df(strains,metabolites, comets_wt, params,filename_wt)
        # 7.- Compute fitness (measure to optimize):
        print('computing fitness...')
        
        # 7.1.- Determine endCycle: when glucose and acetate are exhausted
        iniBiomass, endCycle, totMetSucr_wt = end_simulation_cycle(metabolites[0], strains,df2_wt)
        
        # 7.2.- Compute first element fitness: maximize biomass yield
        finalBiomass_wt =biomass_yield(df2_wt,endCycle, strains)
        if wgMet is not None:

            biomassYieldNew_wt=float(finalBiomass_wt-iniBiomass)/((totMetSucr_wt*wgMet[0]))#+(totMet2*wgMet[1])) # molecular weigth of met per mmol
        # For normalizing yield
        MaximumYield=1
        
        # 7.3.- Compute second element fitnes: minimize time        
        fitTime=1-(float(endCycle)/float(params.all_params['maxCycles']))
        
        # 7.4.- Compute joint fitness, as a 50% each element.
        fitness_wt = compute_fitness(MaximumYield, finalBiomass_wt, iniBiomass, fitFunc, fitTime,
                                 comets_wt, layout_wt,modelcomets_wt,df2_wt)
        # 7.5.- Calculate the uptake of metabolite
        uptakeMet =calculate_uptake(comets_wt,layout_wt, metUptake)
            
            
        print(" Total biomass: "+str(round(finalBiomass_wt,6))+" in cycle "+str(endCycle)+". Biomass yield="+str(round(biomassYieldNew_wt,6)))
   
        totfitness_wt=totfitness_wt+fitness_wt
        fitnessList_wt.append(fitness_wt)
        sumTotBiomass_wt=sumTotBiomass_wt+finalBiomass_wt
        sumTotYield_wt=sumTotYield_wt+biomassYieldNew_wt
            
        # Copy individual solution
        #x= '_'.join(metabolites)
        x=filename_wt
        y='_'.join(metUptake)

            
    avgfitness_wt=totfitness_wt/repeat
    sdfitness_wt=statistics.stdev(fitnessList_wt)
    avgBiomass_wt=sumTotBiomass_wt/repeat
    avgYield_wt=sumTotYield_wt/repeat
    
    print("Fitness_function\tconfiguration\tfitness\tsd\tavg.Biomass\tavg.Yield\tendCycle\tUptakeMet")
                    
    print(fitFunc+"\t"+str(filename_wt)+'\t'+','+y+"\t"+str(round(avgfitness,6))+"\t"+str(sdfitness)+"\t"+str(round(avgBiomass,6))+"\t"+str(round(avgYield,6))+"\t"+str(endCycle)+"\t"+str(uptakeMet))
    with open(dirPlot+"CometsResults"+fitFunc+".txt", "a") as myfile:
        myfile.write("Fitness_function\tconfiguration\tfitness\tsd\tavg.Biomass\tavg.Yield\tendCycle\n")
        myfile.write(fitFunc+"\t"+str(filename)+'\t'+','+"\t"+str(round(avgfitness,6))+"\t"+str(sdfitness)+"\t"+str(round(avgBiomass,6))+"\t"+str(round(avgYield,6))+"\t"+str(endCycle)+"\t"+str(uptakeMet)+"\n")
  
    print("Avg.fitness(sd):\t"+str(avgfitness_wt)+"\t"+str(sdfitness_wt)+"\n")
    if(sdfitness>0.1):
        avgfitness=0.0
    

    
    #Parte conjunta nueva para hacer el gráfico
    
    fitness_separado = [fitness, fitness_wt]
    biomassYieldNew_separado= [biomassYieldNew, biomassYieldNew_wt]
    final_Biomass_separado = [finalBiomass, finalBiomass_wt]
    
    make_graph(strains,metabolites, df2, fitness_separado, biomassYieldNew_separado, final_Biomass_separado, df2_wt, filename)

    
    #tmp models needs to be deleted
    for i in model_id:
        try:
            os.remove(str(i)+'.cmd')
        except FileNotFoundError:
            pass

    return comets.media, comets_wt.media

In [7]:
media, media_wt=Flycop()

This model seems to have metCharge instead of metCharges field. Will use metCharge for what metCharges represents.
No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, p


Initilazing models...
Read LP format model from file /tmp/tmpi2jj8i9d.lp
Reading time = 0.01 seconds
: 2112 rows, 2770 columns, 11744 nonzeros
Parsing metabolites...
Parsing reactions...
Read LP format model from file /tmp/tmp1f319458.lp
Reading time = 0.01 seconds
: 2112 rows, 2770 columns, 11744 nonzeros
Read LP format model from file /tmp/tmprgt7tfg4.lp
Reading time = 0.01 seconds
: 2112 rows, 2770 columns, 11744 nonzeros
Adding monitoring reaction to model...
Read LP format model from file /tmp/tmp498ed6sq.lp
Reading time = 0.01 seconds
: 2113 rows, 2774 columns, 11750 nonzeros
Adding monitoring reaction to model...
Modifications of the model done!
Read LP format model from file /tmp/tmpiqykk5op.lp
Reading time = 0.01 seconds
: 2112 rows, 2770 columns, 11744 nonzeros
Read LP format model from file /tmp/tmppopiaybc.lp
Reading time = 0.01 seconds
: 2112 rows, 2770 columns, 11744 nonzeros
Adding monitoring reaction to model...
Read LP format model from file /tmp/tmpyifa9zd2.lp
Reading

In [3]:
media.loc[media['metabolite']=='acser_e']

,metabolite,cycle,x,y,conc_mmol
